In [3]:
import requests
import json
import time

In [4]:
with open('rulemaker.json', 'r') as file:
    bodyTemplate = json.load(file)

In [5]:
with open('support_rulemaker.json', 'r') as file:
    top40 = json.load(file)

In [6]:
bodyTemplate[2]["parameters"]["min"]

50

In [7]:
bodyTemplate[3]["parameters"]["path"]

'/kg-microbe/abc'

In [ ]:
row_id = 0

for row in top40:
    row_id+=1
    support = int(top40[row]*0.03)

    bodyTemplate[1]["parameters"]["patterns"][0]["head"]["object"]["value"] = "<"+row+">"
    bodyTemplate[1]["parameters"]["thresholds"][0]["value"] = support
    bodyTemplate[2]["parameters"]["min"] = support
    bodyTemplate[3]["parameters"]["path"] = '/kg-microbe/'+row.split("medium/")[1]+'.ndjson'

    URL = "https://precog.vse.cz/rdfrules/api/task"

    body = bodyTemplate

    r = requests.post(url = URL, json = body)

    data = r.json()

    id = data['id']
    print("Odeslal se pozadavek na nalezeni pravidel k mediu ", row, ", task id je ", id, "support je ", support)
    print(bodyTemplate)

    
    #get requests
    getURL = "https://precog.vse.cz/rdfrules/api/task/"+id


    r = requests.get(getURL)

    time.sleep(15)

    while r.status_code == 202:
        r = requests.get(getURL)
        print(row, "(", row_id, "/", len(top40), ")", r.status_code," ... LOGY:", r.json()['logs'])
        time.sleep(15)
    else:
        if r.status_code == 200:
            print("\n Pravidla nalezena k mediu ", row, "(", row_id, "/", len(top40), ")", "\n")
        elif r.status_code != 202 and r.status_code != 200:
            print("\n v mediu ", row, " task ", id, " nastala chyba ", r.status_code, "\n")





Odeslal se pozadavek na nalezeni pravidel k mediu  https://mediadive.dsmz.de/medium/65 , task id je  4a991c8e-9e64-4952-9d9c-1f2e7f9143ee support je  51
[{'name': 'LoadIndex', 'parameters': {'path': '/kg-microbe/kgm_train.index', 'partially': False}}, {'name': 'Mine', 'parameters': {'thresholds': [{'name': 'MinSupport', 'value': 51}, {'name': 'MaxRuleLength', 'value': 4}], 'ruleConsumers': [{'name': 'topK', 'k': 100000, 'allowOverflow': True}], 'patterns': [{'body': [], 'head': {'subject': {'name': 'Variable', 'value': 'a'}, 'predicate': {'name': 'Constant', 'value': 'biolink:occurs_in'}, 'object': {'name': 'Constant', 'value': '<https://mediadive.dsmz.de/medium/65>'}, 'graph': {'name': 'Any'}}, 'exact': False, 'orderless': True}], 'constraints': [{'name': 'WithoutPredicates', 'values': ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', 'rdfs:label']}, {'name': 'OnlyLowerCardinalitySideConstants'}, {'name': 'WithoutDuplicitPredicates'}], 'parallelism': 0}}, {'name': 'ComputeSupport'

KeyboardInterrupt: 